### 네이버 뉴스 크롤링 최종
- 섹션 추출 ~ sub 페이지 추출

#### (1) + (2) + (3)
(1) 섹션 메뉴와 섹션별 url 추출      
    - section, link  
    - section_menu_df  
(2) 섹션별 헤드라인(topic) 제목과 url 추출  
    - topic, url, section  
    - all_topic_df  
(3) 섹션 헤드라인별 sub 페이지 기사 내용 추출  
    - 언론사, 제목, 날짜/시간, 기사내용, 작성자  
    - paper, title, datetime, writing, writer  
    - all_sub_news_info_df   

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import requests

#### (1) 네이버 뉴스 섹션 메뉴와 url 추출

In [3]:
url = 'https://news.naver.com/'
html = urlopen(url)
# 파서 객체 생성
bs_obj = BeautifulSoup(html, 'html.parser')

In [4]:
# 수집된 데이터로 df 구성
section = []
link = []
ul = bs_obj.find('ul', {'class':'Nlnb_menu_list'})
lis = ul.findAll('li')
# 하나씩 추출해서 리스트에 추가
for li in lis:
    a_tag = li.find('a')
    section.append(a_tag.text)
    link.append(a_tag['href'])
section_menu_df = pd.DataFrame({'section':section, 'link':link })
section_menu_df

#### (2) 섹션별 헤드라인(topic) 제목과 url 추출

In [6]:
# 해당 섹션에 대한 모든 topic 크롤링하는 함수 
# 함수로 구성
# 섹션과 url 전달 받아서, 헤드라인 topic의 제목, 링크, 섹션을 추출하여
# 딕셔너리로 만들어 반환하는 함수
def get_topic(url, section):
    # {1} 요청 후 응답 받고 파싱객체 생성
    # 요청 시 전달할 header : 딕셔너리루 구성(key:value, 모두 문자열)
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"}
    res = requests.get(url, headers=headers) # 응답객체를 반환 :해당 객체가 복잡하므로 bs4가 인식할 수 없음
    html = res.text

    # 파싱 객체
    bs_obj=BeautifulSoup(html, "html.parser")
    
    # 파싱
    lis = bs_obj.select("ul.sa_list div.sa_text")

    # 컨텐츠 추출
    title=[]
    titleLink=[]
    for li in lis:
        try:
            title.append(li.a.text.replace("\n", ""))
            titleLink.append(li.a["href"])
        except:
            print("예외 발생")
    return ({"기사제목":title,"기사링크":titleLink, "분류":section})

In [1]:
# 수집된 데이터로 df 구성
all_topic_df = pd.DataFrame({
                "기사제목":[],
                "기사링크":[],
                "분류":[],
                })
# 모든 섹션에 대한 모든 topic 데이터프레임 생성
for i in range(1, 7):
    tmpDF=pd.DataFrame(get_topic(section_menu_df["link"][i], section_menu_df["menu"][i]))
    all_topic_df = pd.concat([all_topic_df, tmpDF], axis=0, ignore_index=True)

#### (3) 섹션 헤드라인별 sub 페이지 기사 내용 추출
- 언론사, 제목, 날짜/시간, 기사내용, 작성자

In [8]:
# sub 페이지 기사 내용 추출하는 함수
def get_sub_news_info(url):
    try:
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"}
        res = requests.get(url, headers=headers)
        html=res.text
        bs_obj=BeautifulSoup(html, "html.parser")

        # 데이터 자장용 빈리스트 생성
        paper_list, title_list, datetime_list, writing_list, writer_list = [[]for _ in range(5)]
        # 데이터 추출
        # 언론사
        paper_list.append(bs_obj.find("a", {"class":"media_end_head_top_logo"}).select_one("img")["title"])
        # 기사제목
        title_list.append(bs_obj.select_one("h2#title_area span").text)
        # 기사입력 날짜 및 시간
        datetime_list.append(bs_obj.select_one(".media_end_head_info_datestamp_time._ARTICLE_DATE_TIME").text)
        # 기사 내용
        writing_list.append(bs_obj.select_one("#dic_area").text.replace("\n", ""))
        # 작성 기자
        if (bs_obj.select_one(".byline_s") == None):
            writer_list.append("없음")
        else:
            writer_list.append(bs_obj.select_one(".byline_s").text)
    except Exception as e:
        print("오류 발생 :", e)
    
    # 지정된 list 내 원소수 일치여부 확인
    print(len(paper_list), len(title_list), len(datetime_list), len(writing_list), len(writer_list))
    sub_new_dict = {"paper":paper_list, 
                                 "title":title_list, 
                                 "datetime":datetime_list, 
                                 "writing":writing_list, 
                                 "writer":writer_list}
    return sub_new_dict

In [3]:
# 모든 sub 페이지 내용 저장하는 데이터프레임 생성해서 최종 데이터프레임 생성
all_sub_news_info_df = pd.DataFrame({"paper":[], 
                                 "title":[], 
                                 "datetime":[], 
                                 "writing":[], 
                                 "writer":[]
                                 })
all_sub_news_info_df
for url in all_topic_df["기사링크"]:
    tmp_df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df=pd.concat([all_sub_news_info_df, tmp_df], axis = 0, ignore_index=True)
# 여기서 나오는 에러값은 요소의 길이중 1이 아닌 것이 있다는 얘기, writer가 공란인 경우가 있음
all_sub_news_info_df

In [12]:
import datetime
today = pd.to_datetime(datetime.datetime.now()).strftime("%Y-%M-%d|%H:%m")
all_sub_news_info_df.to_csv("./crawl_data/naver_sub_news_section_{today}.csv", index=0, encoding="utf-8-sig")

NameError: name 'all_sub_news_info_df' is not defined